In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from geopy.geocoders import Nominatim
from time import sleep
import re

In [ ]:
df = pd.read_csv('accidents_all_regions.csv', sep=';')

In [ ]:
df.head()

,id,tags,category,region,county,address,longitude,latitude,nearby,datetime,light,weather,road_conditions,participants_count,participant_categories,severity,dead_count,injured_count
0,2869171,Дорожно-транспортные происшествия,Наезд на пешехода,Алтайский край,Барнаул,NaN,83.787900,53.391200,Жилые дома индивидуальной застройки,2023-05-24 19:30:00,Светлое время суток,Ясно,Сухое,2,"Все участники, Пешеходы, Мотоциклисты",Легкий,0,1
1,2320091,"Дорожно-транспортные происшествия, ДТП и постр...",Иной вид ДТП,Алтайский край,Барнаул,"г Барнаул, ул Сиреневая, 11",83.666389,53.345833,"Многоквартирные жилые дома, Внутридворовая тер...",2015-09-16 13:00:00,Светлое время суток,Пасмурно,Сухое,2,"Дети, Все участники",Легкий,0,1
2,2321011,"Дорожно-транспортные происшествия, ДТП и постр...",Падение пассажира,Алтайский край,Барнаул,"г Барнаул, ул Попова, 165",83.396880,53.197060,"Административные здания, Остановка общественно...",2017-08-25 15:30:00,Светлое время суток,Ясно,Сухое,3,"Дети, Все участники",Легкий,0,1
3,2576909,Дорожно-транспортные происшествия,Столкновение,Алтайский край,Барнаул,"г Барнаул, ул Петра Сухова, 83 А",83.714200,53.376400,"Выезд с прилегающей территории, Административн...",2021-05-29 19:40:00,Светлое время суток,Ясно,"Сухое, Отсутствие, плохая различимость горизон...",3,"Мотоциклисты, Все участники",Тяжёлый,0,1
4,2547743,Дорожно-транспортные происшествия,Столкновение,Алтайский край,Барнаул,"г Барнаул, Объездная дорога в обход г. Барнаул...",83.708739,53.290476,"Эстакада, путепровод",2021-02-22 13:50:00,Светлое время суток,Ясно,Обработанное противогололедными материалами,4,Все участники,Тяжёлый,0,3


In [ ]:
df.describe()

,id,longitude,latitude,participants_count,dead_count,injured_count
count,1.440208e+06,1.429464e+06,1.429464e+06,1.440208e+06,1.440208e+06,1.440208e+06
mean,2.248653e+06,5.508100e+01,5.373180e+01,2.471898e+00,1.123234e-01,1.265129e+00
std,4.173527e+05,2.676121e+01,5.200464e+00,1.181170e+00,3.875685e-01,8.966765e-01
min,1.527476e+06,-1.791549e+02,3.000000e-04,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.887528e+06,3.791426e+01,5.168444e+01,2.000000e+00,0.000000e+00,1.000000e+00
50%,2.247580e+06,4.454450e+01,5.493050e+01,2.000000e+00,0.000000e+00,1.000000e+00
75%,2.608448e+06,6.071390e+01,5.634739e+01,3.000000e+00,0.000000e+00,1.000000e+00
max,2.978754e+06,1.796408e+02,9.000000e+01,5.900000e+01,2.000000e+01,5.700000e+01


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440208 entries, 0 to 1440207
Data columns (total 18 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   id                      1440208 non-null  int64  
 1   tags                    1440208 non-null  object 
 2   category                1440208 non-null  object 
 3   region                  1440208 non-null  object 
 4   county                  1440208 non-null  object 
 5   address                 1370911 non-null  object 
 6   longitude               1429464 non-null  float64
 7   latitude                1429464 non-null  float64
 8   nearby                  1082922 non-null  object 
 9   datetime                1440208 non-null  object 
 10  light                   1440208 non-null  object 
 11  weather                 1440208 non-null  object 
 12  road_conditions         1440208 non-null  object 
 13  participants_count      1440208 non-null  int64  
 14  pa

In [ ]:
# нужно изменить тип данных datetime

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])

Поиск, анализ и обработка пустых значений

In [ ]:
df.isnull().sum()

,0
id,0
tags,0
category,0
region,0
county,0
address,69297
longitude,10744
latitude,10744
nearby,357286
datetime,0


In [ ]:
# 357 тысяч пустых значений nearby (24,8% от датасета) - информация о том, что находится рядом с местом ДТП, для оценки факторов аварийности несущественна, не будет использована для построения карты, пустые значения оставляем, так как их слишком много- существенные потери, если удалять.
# Данные координат - 10 744 пустые(0,7% от датасета) - несущественная доля пустых значений - менее 1%, пустые значения можем удалить.
# Адрес - 69 297 пустые(4,8% от датасета) - оставвим значения пустыми, так как для нас существеннее данные координат для построения карты.

In [ ]:
df = df.dropna(subset=['longitude', 'latitude'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1429464 entries, 0 to 1440207
Data columns (total 18 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   id                      1429464 non-null  int64         
 1   tags                    1429464 non-null  object        
 2   category                1429464 non-null  object        
 3   region                  1429464 non-null  object        
 4   county                  1429464 non-null  object        
 5   address                 1360801 non-null  object        
 6   longitude               1429464 non-null  float64       
 7   latitude                1429464 non-null  float64       
 8   nearby                  1077366 non-null  object        
 9   datetime                1429464 non-null  datetime64[ns]
 10  light                   1429464 non-null  object        
 11  weather                 1429464 non-null  object        
 12  road_conditions    

Поиск и удаление дубликатов

In [ ]:
full_duplicates = df[df.duplicated(keep=False)]
full_duplicates

,id,tags,category,region,county,address,longitude,latitude,nearby,datetime,light,weather,road_conditions,participants_count,participant_categories,severity,dead_count,injured_count


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
# Найти частичные дубликаты (одинаковый id)
partial_duplicates = df[df.duplicated(subset=["id"], keep=False)]
partial_duplicates

,id,tags,category,region,county,address,longitude,latitude,nearby,datetime,light,weather,road_conditions,participants_count,participant_categories,severity,dead_count,injured_count


In [ ]:
# дубликатов в датасете не обнаружено

Проверяем корректность значений ширины и долготы: посмотреть min и max значения по регионам, есть ли выбросы

In [ ]:
unique_count = df['region'].nunique()
unique_count

85

In [ ]:
coord_regions = df.groupby('region').agg(
    min_longitude=('longitude', 'min'),
    max_longitude=('longitude', 'max'),
    min_latitude=('latitude', 'min'),
    max_latitude=('latitude', 'max')
).reset_index()

coord_regions

,region,min_longitude,max_longitude,min_latitude,max_latitude
0,Алтайский край,1.000000,90.520000,1.000000,86.951389
1,Амурская область,0.253229,176.432500,0.127132,84.254600
2,Архангельская область,32.000000,75.762500,0.005000,71.557200
3,Астраханская область,0.284700,85.000000,3.000000,55.571700
4,Белгородская область,3.000000,65.002200,5.000000,90.000000
...,...,...,...,...,...
80,Чеченская Республика,0.100000,81.579200,4.381900,85.000000
81,Чувашская Республика - Чувашия,0.013100,60.941700,0.015600,66.038100
82,Чукотский автономный округ,-179.154910,179.640841,17.700300,74.108100
83,Ямало-Ненецкий автономный округ,48.488100,97.004400,6.082200,75.818900


In [ ]:
coord_regions

,region,min_longitude,max_longitude,min_latitude,max_latitude
0,Алтайский край,1.000000,90.520000,1.000000,86.951389
1,Амурская область,0.253229,176.432500,0.127132,84.254600
2,Архангельская область,32.000000,75.762500,0.005000,71.557200
3,Астраханская область,0.284700,85.000000,3.000000,55.571700
4,Белгородская область,3.000000,65.002200,5.000000,90.000000
...,...,...,...,...,...
80,Чеченская Республика,0.100000,81.579200,4.381900,85.000000
81,Чувашская Республика - Чувашия,0.013100,60.941700,0.015600,66.038100
82,Чукотский автономный округ,-179.154910,179.640841,17.700300,74.108100
83,Ямало-Ненецкий автономный округ,48.488100,97.004400,6.082200,75.818900


Минимальные и максимальные значения географических координат в целом:

Широта — от −90° до +90°.

Долгота — от −180° до +180°

In [ ]:
errors = []

# границы для долготы и широты
long_error = coord_regions[(coord_regions['min_longitude'] < -180) |
                            (coord_regions['max_longitude'] > 180) |
                            (coord_regions['min_longitude'] > coord_regions['max_longitude'])]
if not long_error.empty:
    errors.append(("Долгота", long_error))

lat_error = coord_regions[(coord_regions['min_latitude'] < -90) |
                          (coord_regions['max_latitude'] > 90) |
                          (coord_regions['min_latitude'] > coord_regions['max_latitude'])]
if not lat_error.empty:
    errors.append(("Широта", lat_error))

# Выводим ошибки
if errors:
    for error_type, error_data in errors:
        print(f"Ошибки в {error_type}:")
        print(error_data[['region', 'min_longitude' if error_type == 'Долгота' else 'min_latitude',
                                   'max_longitude' if error_type == 'Долгота' else 'max_latitude']])
else:
    print("Значения координат соответствуют минимальным и максимальным значениям долготы и ширины всего по миру.")

Значения координат соответствуют минимальным и максимальным значениям долготы и ширины всего по миру.


Минимальные и максимальные значения географических координат в РФ:

Широта — от 41° до 82°.

Долгота — от 19° до 170°.

In [ ]:
# проверяем на соответствие координат min и max значениях долготы и широты в пределах РФ
errors = []

long_error = coord_regions[(coord_regions['min_longitude'] < 19) |
                            (coord_regions['max_longitude'] > 170) |
                            (coord_regions['min_longitude'] > coord_regions['max_longitude'])]
if not long_error.empty:
    errors.append(("Долгота", long_error))

lat_error = coord_regions[(coord_regions['min_latitude'] < 41) |
                          (coord_regions['max_latitude'] > 82) |
                          (coord_regions['min_latitude'] > coord_regions['max_latitude'])]
if not lat_error.empty:
    errors.append(("Широта", lat_error))

# Выводим ошибки
if errors:
    for error_type, error_data in errors:
        print(f"Ошибки в {error_type}:")
        print(error_data[['region', 'min_longitude' if error_type == 'Долгота' else 'min_latitude',
                                   'max_longitude' if error_type == 'Долгота' else 'max_latitude']])
else:
    print("Ошибок не найдено.")

Ошибки в Долгота:
                            region  min_longitude  max_longitude
0                   Алтайский край       1.000000      90.520000
1                 Амурская область       0.253229     176.432500
3             Астраханская область       0.284700      85.000000
4             Белгородская область       3.000000      65.002200
5                 Брянская область       3.318889     103.812882
..                             ...            ...            ...
79             Челябинская область       0.201100     113.682800
80            Чеченская Республика       0.100000      81.579200
81  Чувашская Республика - Чувашия       0.013100      60.941700
82      Чукотский автономный округ    -179.154910     179.640841
84             Ярославская область       0.010600      58.353300

[78 rows x 3 columns]
Ошибки в Широта:
                             region  min_latitude  max_latitude
0                    Алтайский край      1.000000     86.951389
1                  Амурская област

Мы наблюдаем большое количество ошибок в данных в части корректности указания координат мест ДТП - они выходят за рамки min и max значений для субъектов РФ.

In [ ]:
count_errors_coord = df[(df['longitude'] < 19) |
                  (df['longitude'] > 170) |
                  (df['latitude'] < 41) |
                  (df['latitude'] > 82)].shape[0]

print(f"Количество записей с ошибочными координатами: {count_errors_coord}")

Количество записей с ошибочными координатами: 4739


Количество строк с ошибочными координатами - 4 739 или 0,33% от всех записей о ДТП, ввиду несущественности можем удалить эти строки.

In [ ]:
correct_coords_mask = (
    (df['longitude'] >= 19) & (df['longitude'] <= 170) &
    (df['latitude'] >= 41) & (df['latitude'] <= 82)
)

# отфильтровываем, оставляем только корректные строки
df_clean = df[correct_coords_mask]

count_errors_coord = df.shape[0] - df_clean.shape[0]
df = df_clean
print(f"Количество удалённых записей с ошибочными координатами: {count_errors_coord}")


Количество удалённых записей с ошибочными координатами: 4739


In [ ]:
#проверка
count_errors_coord = df[(df['longitude'] < 19) |
                  (df['longitude'] > 170) |
                  (df['latitude'] < 41) |
                  (df['latitude'] > 82)].shape[0]

print(f"Количество записей с ошибочными координатами: {count_errors_coord}")

Количество записей с ошибочными координатами: 0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1424725 entries, 0 to 1440207
Data columns (total 18 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   id                      1424725 non-null  int64         
 1   tags                    1424725 non-null  object        
 2   category                1424725 non-null  object        
 3   region                  1424725 non-null  object        
 4   county                  1424725 non-null  object        
 5   address                 1356286 non-null  object        
 6   longitude               1424725 non-null  float64       
 7   latitude                1424725 non-null  float64       
 8   nearby                  1074540 non-null  object        
 9   datetime                1424725 non-null  datetime64[ns]
 10  light                   1424725 non-null  object        
 11  weather                 1424725 non-null  object        
 12  road_conditions    

Упрощаем значения некоторых колонок, укрупняем данные

In [ ]:
df['tags'].unique()

array(['Дорожно-транспортные происшествия',
       'Дорожно-транспортные происшествия, ДТП и пострадавшие дети в возрасте до 16 лет',
       'Дорожно-транспортные происшествия, ДТП и пострадавшие пешеходы, из-за наезда на пешеходов, на пешеходных переходах, по вине пешеходов',
       'Дорожно-транспортные происшествия, ДТП и пострадавшие пешеходы, из-за наезда на пешеходов, на пешеходных переходах, по вине пешеходов, ДТП и пострадавшие дети в возрасте до 16 лет',
       'Дорожно-транспортные происшествия, ДТП и пострадавшие дети в возрасте до 16 лет, ДТП и пострадавшие пешеходы, из-за наезда на пешеходов, на пешеходных переходах, по вине пешеходов'],
      dtype=object)

In [ ]:
value_1 = 'Дорожно-транспортные происшествия, ДТП и пострадавшие пешеходы, из-за наезда на пешеходов, на пешеходных переходах, по вине пешеходов, ДТП и пострадавшие дети в возрасте до 16 лет'
value_2 = 'Дорожно-транспортные происшествия, ДТП и пострадавшие дети в возрасте до 16 лет, ДТП и пострадавшие пешеходы, из-за наезда на пешеходов, на пешеходных переходах, по вине пешеходов'

new_value = 'ДТП на пешеходном переходе с пострадавшими до 16 лет (наезд по вине пешехода)'

df['tags'] = df['tags'].replace(
    [value_1, value_2],
    new_value
)

In [ ]:
value_3 = 'Дорожно-транспортные происшествия, ДТП и пострадавшие дети в возрасте до 16 лет'
new_value1 = 'ДТП с подстрадавшими до 16 лет'

df['tags'] = df['tags'].replace(
    [value_3],
    new_value1
)

In [ ]:
value_4 = 'Дорожно-транспортные происшествия, ДТП и пострадавшие пешеходы, из-за наезда на пешеходов, на пешеходных переходах, по вине пешеходов'
new_value2 = 'ДТП на пешеходном переходе с пострадавшими (наезд по вине пешехода)'

df['tags'] = df['tags'].replace(
    [value_4],
    new_value2
)

In [ ]:
value_5 = 'Дорожно-транспортные происшествия'
new_value3 = 'ДТП'

df['tags'] = df['tags'].replace(
    [value_5],
    new_value3
)

In [ ]:
df['category'].unique()

array(['Наезд на пешехода', 'Иной вид ДТП', 'Падение пассажира',
       'Столкновение', 'Наезд на велосипедиста', 'Съезд с дороги',
       'Наезд на препятствие', 'Наезд на стоящее ТС', 'Опрокидывание',
       'Наезд на животное',
       'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее производство работ',
       'Отбрасывание предмета',
       'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее несение службы',
       'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее какую-либо другую деятельность',
       'Наезд на внезапно возникшее препятствие',
       'Наезд на гужевой транспорт',
       'Возгорание вследствие технической неисправности движущегося или остановившегося ТС, участвующего в дорожном движении.',
       'Падение груза'], dtype=object)

In [ ]:
value_c1 = 'Возгорание вследствие технической неисправности движущегося или остановившегося ТС, участвующего в дорожном движении.'
new_value_c1 = 'Возгорание ТС'

df['category'] = df['category'].replace(
    [value_c1],
    new_value_c1
)

In [ ]:
value_c2 = 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее производство работ'
value_c3 = 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее несение службы'
value_c4 = 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее какую-либо другую деятельность'

new_value_с2 = 'Наезд на пешехода'

df['category'] = df['category'].replace(
    [value_c2, value_c3, value_c4],
    new_value_с2
)

In [ ]:
df['nearby'].unique()

array(['Жилые дома индивидуальной застройки',
       'Многоквартирные жилые дома, Внутридворовая территория, Школа либо иное детское (в т.ч. дошкольное) учреждение',
       'Административные здания, Остановка общественного транспорта, Нерегулируемый пешеходный переход, Автостоянка (отделенная от проезжей части), Остановка трамвая',
       ...,
       'Зоны отдыха, Жилые дома индивидуальной застройки, Объект (здание, сооружение) религиозного культа, Аэропорт, ж/д вокзал (ж/д станция), речной или морской порт (пристань)',
       'Многоквартирные жилые дома, Одиночный торговый объект, являющийся местом притяжения транспорта и (или) пешеходов, Остановка маршрутного такси, Школа либо иное детское (в т.ч. дошкольное) учреждение, Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иного детского учреждения',
       'Крупный торговый объект (являющийся объектом массового тяготения пешеходов и (или) транспорта), Школа либо иная дет

In [ ]:
unique_count = df['nearby'].nunique()
unique_count

48339

In [ ]:
df['nearby_all'] = df['nearby']

In [ ]:
def auto_grouping_nearby(value):
    if not isinstance(value, str):
        return value

    value_lower = value.lower()
    parts = [part.strip() for part in value_lower.split(',')]

    # Список условий в порядке приоритета
    conditions = [
        (["нерегулируемый перекресток", "нерегулируемый перекрёсток"], "Перекресток"),  # Нерегулируемый перекресток
        (["регулируемый перекресток"], "Перекресток"),
        (["регулируемый перекрёсток"], "Перекресток"),
        (["нерегулируемый пешеходный переход"], "Пешеходный переход"),
        (["регулируемый пешеходный переход"], "Пешеходный переход"),
        (["надземный пешеходный переход"], "Пешеходный переход"),
        (["подземный пешеходный переход"], "Пешеходный переход"),
        (["пешеходная зона", "тротуар", "пешеходная дорожка"], "Пешеходная зона"),
        (["с круговым движением"], "Перекресток"),
        (["остановка"], "Остановка"),
        (["ж/д переезд"], "Ж/д переезд"),
        (["мост", "эстакад", "путепровод"], "Мосты, эстакады, путепроводы"),
        (["тоннель"], "Тоннели"),
        (["жит", "жилые", "внутридворовая"], "Жилые дома"),
        (["школа", "детский", "дошкольное", "образовательн"], "Школы и садики"),
        (["административные", "офисы", "мвд", "дпс", "ппс", "религиозн"], "Административные здания"),
        (["торговый", "магазин", "торговый объект", "коммерческий", "торговли", "производственное предприятие", "объект строительства", "азс"], "Коммерческие объекты"),
        (["спортивный", "развлекательный", "отдыха"], "Развлекательные объекты"),
        (["выезд с прилегающей территории"], "Выезд с прилегающей территории"),
        (["медицинский", "лечебные"], "Медицинские организации"),
        (["автовокзал", "вокзал", "аэропорт"], "Вокзалы и аэропорты"),
        (["автостоянка", "стоянка", "парковк"], "Автостоянки")
    ]

    first_match = None

    for part in parts:
        for keywords, category in conditions:
            if any(word in part for word in keywords):
                if first_match is None:
                    first_match = category
                break
    return first_match if first_match is not None else "Иные объекты"

df['nearby'] = df['nearby'].apply(auto_grouping_nearby)

In [ ]:
unique_count = df['nearby'].nunique()
unique_count

17

Сгруппировали данные до 17 видов (было 48 тысяч) в колонке 'nearby'- пока оставим в таком виде, возможно, позднее еще укрупним.

In [ ]:
df.head(500)

,id,tags,category,region,county,address,longitude,latitude,nearby,datetime,light,weather,road_conditions,participants_count,participant_categories,severity,dead_count,injured_count,nearby_all
0,2869171,ДТП,Наезд на пешехода,Алтайский край,Барнаул,NaN,83.787900,53.391200,Жилые дома,2023-05-24 19:30:00,Светлое время суток,Ясно,Сухое,2,"Все участники, Пешеходы, Мотоциклисты",Легкий,0,1,Жилые дома индивидуальной застройки
1,2320091,ДТП с подстрадавшими до 16 лет,Иной вид ДТП,Алтайский край,Барнаул,"г Барнаул, ул Сиреневая, 11",83.666389,53.345833,Жилые дома,2015-09-16 13:00:00,Светлое время суток,Пасмурно,Сухое,2,"Дети, Все участники",Легкий,0,1,"Многоквартирные жилые дома, Внутридворовая тер..."
2,2321011,ДТП с подстрадавшими до 16 лет,Падение пассажира,Алтайский край,Барнаул,"г Барнаул, ул Попова, 165",83.396880,53.197060,Административные здания,2017-08-25 15:30:00,Светлое время суток,Ясно,Сухое,3,"Дети, Все участники",Легкий,0,1,"Административные здания, Остановка общественно..."
3,2576909,ДТП,Столкновение,Алтайский край,Барнаул,"г Барнаул, ул Петра Сухова, 83 А",83.714200,53.376400,Выезд с прилегающей территории,2021-05-29 19:40:00,Светлое время суток,Ясно,"Сухое, Отсутствие, плохая различимость горизон...",3,"Мотоциклисты, Все участники",Тяжёлый,0,1,"Выезд с прилегающей территории, Административн..."
4,2547743,ДТП,Столкновение,Алтайский край,Барнаул,"г Барнаул, Объездная дорога в обход г. Барнаул...",83.708739,53.290476,"Мосты, эстакады, путепроводы",2021-02-22 13:50:00,Светлое время суток,Ясно,Обработанное противогололедными материалами,4,Все участники,Тяжёлый,0,3,"Эстакада, путепровод"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2556446,ДТП,Наезд на пешехода,Алтайский край,Барнаул,"г Барнаул, ул Советской Армии, 171А",83.703815,53.356001,Жилые дома,2021-03-04 11:35:00,Светлое время суток,Пасмурно,"Отсутствие, плохая различимость горизонтальной...",2,"Пешеходы, Все участники",Тяжёлый,0,1,"Многоквартирные жилые дома, Жилые дома индивид..."
496,2313659,ДТП,Столкновение,Алтайский край,Барнаул,"г Барнаул, пр-кт Космонавтов, 18а",83.682368,53.387309,Выезд с прилегающей территории,2020-01-20 08:30:00,Светлое время суток,Пасмурно,"Сухое, Недостатки зимнего содержания",4,Все участники,Легкий,0,1,"Выезд с прилегающей территории, Административн..."
497,2322934,ДТП,Наезд на пешехода,Алтайский край,Барнаул,"г Барнаул, пр-кт Космонавтов, 14",83.414700,53.231400,Коммерческие объекты,2017-03-27 05:50:00,"В темное время суток, освещение включено","Пасмурно, Дождь","Мокрое, Отсутствие, плохая различимость горизо...",2,"Пешеходы, Все участники",Легкий,0,1,"Производственное предприятие, Нерегулируемый п..."
498,2556448,ДТП,Наезд на пешехода,Алтайский край,Барнаул,NaN,83.700800,53.362800,Жилые дома,2021-03-03 12:30:00,Светлое время суток,Пасмурно,Со снежным накатом,2,"Пешеходы, Все участники",Легкий,0,1,"Многоквартирные жилые дома, Внутридворовая тер..."


Создадим новую колонку с данными о температуре

In [ ]:
def extract_temp_info(weather_str):
    weather_lower = weather_str.lower()
    if 'температура ниже -30' in weather_lower:
        return 'Температура ниже -30С'
    elif 'температура выше +30' in weather_lower:
        return 'Температура выше +30С'
    else:
        return 'нет данных'

df['temp'] = df['weather'].apply(extract_temp_info)

In [ ]:
df['temp'].unique()

array(['нет данных', 'Температура ниже -30С', 'Температура выше +30С'],
      dtype=object)

In [ ]:
def auto_grouping_weather(value):
    if not isinstance(value, str):
        return value

    value_lower = value.lower()
    parts = [part.strip() for part in value_lower.split(',')]

    # Список условий в порядке приоритета
    conditions = [
        (["дождь"], "Дождь"),
        (["снегопад"], "Снегопад"),
        (["туман"], "Туман"),
        (["метель"], "Метель"),
        (["ураган"], "Ураганный ветер"),
        (["пасмурно"], "Пасмурно"),
        (["ясно"], "Ясно")
    ]

    first_match = None

    for part in parts:
        for keywords, category in conditions:
            if any(word in part for word in keywords):
                if first_match is None:
                    first_match = category
                break

    return first_match if first_match is not None else "нет данных"

df['weather'] = df['weather'].apply(auto_grouping_weather)

In [ ]:
df['weather'].unique()

array(['Ясно', 'Пасмурно', 'Снегопад', 'Дождь', 'Метель', 'Туман'],
      dtype=object)

In [ ]:
df['road_conditions_all'] = df['road_conditions']

Выведем новую колонку "wet_road" для обозначения состояния дорожного покрытия (влажность и загрязненность).

In [ ]:
def extract_wet_road_info(road_str):
    if not isinstance(road_str, str):
        return 'нет данных'

    road_lower = road_str.lower()

    if 'сухое' in road_lower:
        return 'Сухое'
    elif 'мокрое' in road_lower or 'залитое' in road_lower:
        return 'Мокрое'
    elif 'гололедица' in road_lower:
        return 'Гололедица'
    elif 'заснежен' in road_lower or 'снежным' in road_lower:
        return 'Заснеженное'
    elif 'загрязнен' in road_lower or 'пыльное' in road_lower:
        return 'Мокрое'           # загрязненное
    else:
        return 'нет данных'

df['wet_road'] = df['road_conditions'].apply(extract_wet_road_info)

Выведем новую колонку "road_light" для обозначения наличия или отсутствия освещения дороги.

In [ ]:
def extract_road_light(road_str):
    if not isinstance(road_str, str):
        return 'нет данных'

    road_lower = road_str.lower()

    if 'отсутствие освещения' in road_lower or 'недостаточное освещение' in road_lower or 'неисправное освещение' in road_lower:
        return 'Освещение отсутствует'
    else:
        return 'нет данных'

df['road_light'] = df['road_conditions'].apply(extract_road_light)

In [ ]:
# Заменяем "Нет данных" на "Освещение отсутствует", если в 'light' есть "освещение отсутствует"
df.loc[(df['road_light'] == 'нет данных') & (df['light'].str.contains('освещение отсутствует', na=False, case=False)), 'road_light'] = 'Освещение отсутствует'
# Заменяем "Нет данных" на "Освещение есть", если в 'light' есть "освещение включено" или "освещение не включено"
df.loc[(df['road_light'] == 'нет данных') & (df['light'].str.contains('освещение включено', na=False, case=False)), 'road_light'] = 'Освещение есть'
df.loc[(df['road_light'] == 'нет данных') & (df['light'].str.contains('освещение не включено', na=False, case=False)), 'road_light'] = 'Освещение есть'

Выведем новую колонку "road_treatment" для обозначения обработки дороги.

In [ ]:
def extract_road_treatment(road_str):
    if not isinstance(road_str, str):
        return 'нет данных'

    road_lower = road_str.lower()

    if 'обработанное противогололедными' in road_lower:
        return 'Обработано от льда'
    elif 'свежеуложенная поверхностная обработка' in road_lower:
        return 'Свежеуложеная обработка'
    elif 'недостатки зимнего содержания' in road_lower:
        return 'Обработка отсутствует'
    else:
        return 'нет данных'

df['road_treatment'] = df['road_conditions'].apply(extract_road_treatment)

Выведем новую колонку "road_signs" для обозначения видимости дорожных знаков и разметки, их наличия/отсутствия.

In [ ]:
def extract_road_signs(road_str):
    if not isinstance(road_str, str):
        return 'нет данных'

    road_lower = road_str.lower()

    if 'отсутствие дорожных знаков' in road_lower or 'отсутствие дорожных ограждений' in road_lower or 'отсутствие временных тсод' in road_lower or 'несоответствие дорожных ограждений' in road_lower or 'отсутствие пешеходных ограждений' in road_lower or 'отсутствие элементов обустройства остановочного' in road_lower:
        return 'Отсутствие дорожных знаков или ограждений'
    elif 'плохая различимость вертикальной разметки' in road_lower or 'плохая различимость горизонтальной разметки' in road_lower or 'отсутствие направляющих устройств' in road_lower:
        return 'Дефекты разметки'
    elif 'плохая видимость светофора' in road_lower or 'плохая видимость дорожных знаков' in road_lower or 'нарушения в размещении наружной' in road_lower or 'ограничение видимости' in road_lower:
        return 'Плохая видимость светофора или знаков'
    elif 'неисправность светофора' in road_lower:
        return 'Неисправность светофора'
    else:
        return 'нет данных'

df['road_signs'] = df['road_conditions'].apply(extract_road_signs)

Теперь в колонке df['road_conditions'] останутся только дефекты дорожного покрытия, сгруппируем информацию.

In [ ]:
unique_count = df['road_conditions'].nunique()
unique_count

7153

In [ ]:
def auto_grouping_road(value):
    if not isinstance(value, str):
        return value

    value_lower = value.lower()
    parts = [part.strip() for part in value_lower.split(',')]

    conditions = [
        (["дефекты покрытия", "неровное покрытие", "неровное покрытие", "низкие сцепные качества", "неудовлетворительное состояние обочин", "несоответствие люков"], "Дефекты покрытия"),
        (["отсутствие тротуаров"], "Отсутствие тротуаров"),
        (["несоответствие железнодорожного переезда", "отклонение верха головки рельса"], "Дефекты ж/д переезда и рельсов"),
        (["сужение"], "Сужение дороги"),
        (["иные недостатки"], "Иные недостатки")
    ]

    first_match = None

    for part in parts:
        for keywords, category in conditions:
            if any(word in part for word in keywords):
                if first_match is None:
                    first_match = category
                break

    return first_match if first_match is not None else "Иные недостатки"

df['road_conditions'] = df['road_conditions'].apply(auto_grouping_road)

In [ ]:
unique_count = df['road_conditions'].nunique()
unique_count

5

In [ ]:
df['road_conditions'].unique()

array(['Иные недостатки', 'Дефекты покрытия', 'Отсутствие тротуаров',
       'Дефекты ж/д переезда и рельсов', 'Сужение дороги'], dtype=object)

In [ ]:
columns = ["light",'road_conditions', 'wet_road', 'road_light', "road_treatment", 'road_signs']

selected_df = df[columns]

selected_df.head(500)

,light,road_conditions,wet_road,road_light,road_treatment,road_signs
0,Светлое время суток,Иные недостатки,Сухое,нет данных,нет данных,нет данных
1,Светлое время суток,Иные недостатки,Сухое,нет данных,нет данных,нет данных
2,Светлое время суток,Иные недостатки,Сухое,нет данных,нет данных,нет данных
3,Светлое время суток,Иные недостатки,Сухое,нет данных,нет данных,Дефекты разметки
4,Светлое время суток,Иные недостатки,нет данных,нет данных,Обработано от льда,нет данных
...,...,...,...,...,...,...
495,Светлое время суток,Иные недостатки,Заснеженное,нет данных,Обработка отсутствует,Дефекты разметки
496,Светлое время суток,Иные недостатки,Сухое,нет данных,Обработка отсутствует,нет данных
497,"В темное время суток, освещение включено",Иные недостатки,Мокрое,Освещение есть,нет данных,Отсутствие дорожных знаков или ограждений
498,Светлое время суток,Иные недостатки,Заснеженное,нет данных,нет данных,нет данных


In [ ]:
unique_count = df['participant_categories'].nunique()
unique_count

55

In [ ]:
df['participant_categories'].unique()

array(['Все участники, Пешеходы, Мотоциклисты', 'Дети, Все участники',
       'Мотоциклисты, Все участники', 'Все участники',
       'Пешеходы, Дети, Все участники', 'Велосипедисты, Все участники',
       'Пешеходы, Все участники', 'Велосипедисты, Дети, Все участники',
       'Мотоциклисты, Дети, Все участники',
       'Велосипедисты, Мотоциклисты, Все участники',
       'Пешеходы, Общ. транспорт, Все участники',
       'Общ. транспорт, Все участники',
       'Пешеходы, Мотоциклисты, Дети, Все участники',
       'Пешеходы, Мотоциклисты, Все участники', 'Все участники, Пешеходы',
       'Общ. транспорт, Дети, Все участники',
       'Все участники, Велосипедисты, Мотоциклисты',
       'Дети, Пешеходы, Все участники', 'Все участники, Велосипедисты',
       'Все участники, Общ. транспорт', 'Все участники, Дети',
       'Все участники, Мотоциклисты',
       'Все участники, Пешеходы, Общ. транспорт',
       'Мотоциклисты, Общ. транспорт, Все участники',
       'Велосипедисты, Мотоциклисты, Д

In [ ]:
df_exploded = df['participant_categories'].str.split(', ').explode()

df_exploded.unique()

array(['Все участники', 'Пешеходы', 'Мотоциклисты', 'Дети',
       'Велосипедисты', 'Общ. транспорт'], dtype=object)

In [ ]:
df_participants_encoded = df['participant_categories'].str.get_dummies(sep=', ').fillna(0)

df_participants_encoded

,Велосипедисты,Все участники,Дети,Мотоциклисты,Общ. транспорт,Пешеходы
0,0,1,0,1,0,1
1,0,1,1,0,0,0
2,0,1,1,0,0,0
3,0,1,0,1,0,0
4,0,1,0,0,0,0
...,...,...,...,...,...,...
1440203,0,1,0,0,0,0
1440204,0,1,0,0,0,0
1440205,0,1,0,0,0,0
1440206,1,1,1,0,0,0


In [ ]:
df = pd.concat([df, df_participants_encoded], axis=1)

In [ ]:
def normalize_categories_without_all(category_string):
    if isinstance(category_string, str):
        categories = [cat.strip() for cat in category_string.split(',') if cat.strip()]
        filtered_categories = [cat for cat in categories if cat != 'Все участники']

        if not filtered_categories:
             return 'Только водители ТС'
        normalized_categories = sorted(list(set(filtered_categories)))
        return ', '.join(normalized_categories)
    else:
        return None

df['participant_categories_normalized'] = df['participant_categories'].apply(normalize_categories_without_all)

df[['participant_categories', 'participant_categories_normalized']]


,participant_categories,participant_categories_normalized
0,"Все участники, Пешеходы, Мотоциклисты","Мотоциклисты, Пешеходы"
1,"Дети, Все участники",Дети
2,"Дети, Все участники",Дети
3,"Мотоциклисты, Все участники",Мотоциклисты
4,Все участники,Только водители ТС
...,...,...
1440203,Все участники,Только водители ТС
1440204,Все участники,Только водители ТС
1440205,Все участники,Только водители ТС
1440206,"Велосипедисты, Дети, Все участники","Велосипедисты, Дети"


In [ ]:
print("\nГруппировка по комбинациям:")
df.groupby('participant_categories_normalized').size().reset_index(name='Количество_ДТП')


Группировка по комбинациям:


,participant_categories_normalized,Количество_ДТП
0,Велосипедисты,35687
1,"Велосипедисты, Дети",11989
2,"Велосипедисты, Дети, Мотоциклисты",588
3,"Велосипедисты, Дети, Мотоциклисты, Пешеходы",4
4,"Велосипедисты, Дети, Общ. транспорт",21
5,"Велосипедисты, Дети, Пешеходы",62
6,"Велосипедисты, Мотоциклисты",905
7,"Велосипедисты, Мотоциклисты, Пешеходы",3
8,"Велосипедисты, Общ. транспорт",89
9,"Велосипедисты, Общ. транспорт, Пешеходы",1


In [ ]:
df['participant_categories']= df['participant_categories_normalized']
df = df.drop('participant_categories_normalized', axis=1)

In [ ]:
df = df[df['id'] != 2947689]           #при открытии файла выяснилось, что строка с данным индектом пустая - удалим ее

In [ ]:
file_path = r'accidents_all_regions_04-05.csv'
df.to_csv(file_path, index=False)

Приступаем к обработке файла с участниками ДТП

In [ ]:
df1 = pd.read_csv('participants_all_regions.csv', sep=';')

In [ ]:
# удалим строки про участников ДТП, которые были удалены при обработке файла accidents

In [ ]:
accident_ids = df['id'].unique()
df1 = df1[df1['accident_id'].isin(accident_ids)].copy()

In [ ]:
df1.head()

,accident_id,vehicle_id,participant_id,role,gender,violations,health_status,years_of_driving_experience
0,2869171,2869171_1,2869171_1_1,Водитель,Мужской,"Управление мопедом лицом моложе 16 лет, Управл...",Не пострадал,NaN
1,2869171,NaN,2869171_0_1,Пешеход,Мужской,NaN,"Раненый, находящийся (находившийся) на амбулат...",NaN
2,2320091,2320091_1,2320091_1_1,Водитель,Мужской,"Несоблюдение условий, разрешающих движение тра...",Не пострадал,14.0
3,2321011,2321011_1,2321011_1_1,Водитель,Мужской,NaN,Не пострадал,25.0
4,2321011,2321011_1,2321011_1_2,Пассажир,Женский,Нарушение правил пользования общественным тран...,Не пострадал,NaN


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3511232 entries, 0 to 3547229
Data columns (total 8 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   accident_id                  int64  
 1   vehicle_id                   object 
 2   participant_id               object 
 3   role                         object 
 4   gender                       object 
 5   violations                   object 
 6   health_status                object 
 7   years_of_driving_experience  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 241.1+ MB


In [ ]:
df1.isnull().sum()

,0
accident_id,0
vehicle_id,444764
participant_id,0
role,0
gender,92392
violations,1795985
health_status,6273
years_of_driving_experience,1553729


In [ ]:
# пустые значения vehicle_id, years_of_driving_experience	вероятнее всего означают пешехода/пассажира (ниже проверим),
# пустые значения violations означают, что участник не нарушал ПДД,
# в колонке пол пустых значений 92 392 или 2,6% от всех участников ДТП - можно удалить пустые строки,
# в колонке health_status пустых значений 6 273 или 0,18% от всех участников ДТП - можно удалить пустые строки

In [ ]:
df1 = df1.dropna(subset=['gender', 'health_status'])

In [ ]:
df1['role'].unique()

array(['Водитель', 'Пешеход', 'Пассажир', 'Велосипедист',
       'Пешеход, перед ДТП находившийся в (на) ТС в качестве водителя или пешеход, перед ДТП находившийся в (на) ТС в качестве пассажира'],
      dtype=object)

In [ ]:
role_1 = 'Пешеход, перед ДТП находившийся в (на) ТС в качестве водителя или пешеход, перед ДТП находившийся в (на) ТС в качестве пассажира'

new_role = 'Пешеход'

df1['role'] = df1['role'].replace(
    [role_1], new_role
)

In [ ]:
df1['health_status'].unique()

array(['Не пострадал',
       'Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара',
       'Раненый, находящийся (находившийся)  на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)',
       'Раненый, находящийся (находившийся) на стационарном лечении',
       'Получил телесные повреждения с показанием к лечению в медицинских организациях (кроме разовой медицинской помощи)',
       'Скончался в течение 1 суток',
       'Скончался на месте ДТП до приезда скорой медицинской помощи',
       'Получил травмы с оказанием разовой медицинской помощи, к категории раненый не относится',
       'Скончался в течение 12 суток', 'Скончался в течение 28 суток',
       'Скончался при транспортировке', 'Скончался в течение 2 суток',
       'Скончался в течение 16 суток', 'Скончался в течение 4 суток',
       'Скончался на месте ДТП по прибытию ск

In [ ]:
def categorize_health_status_in_place(df1):
    try:
        df1['health_status'] = df1['health_status'].astype(str)

        def categorize(text):
            if 'Скончался' in text:
                return 'Скончался'
            elif 'Не пострадал' in text:
                return 'Не пострадал'
            elif 'Получил травмы с оказанием разовой медицинской помощи, к категории раненый не относится' in text:
                return 'Легкая травма'
            elif 'Получил телесные повреждения с показанием к лечению в медицинских организациях (кроме разовой медицинской помощи)' in text or \
                 'Получил телесные повреждения с показанием к лечению в медицинских организациях, фактически лечение не проходил, к категории раненый не относится' in text:
                return 'Травма средней тяжести'
            elif 'Раненый, наход' in text:
                return 'Тяжелая травма'
            else:
                return text

        df1['health_status'] = df1['health_status'].apply(categorize)
    except (KeyError, TypeError) as e:
        print(f"Ошибка: {e}")

categorize_health_status_in_place(df1)

In [ ]:
df1['health_status'].unique()

array(['Не пострадал', 'Тяжелая травма', 'Травма средней тяжести',
       'Скончался', 'Легкая травма'], dtype=object)

In [ ]:
drivers_df = df1[df1['role'] == 'Водитель']
years_of_experience_for_drivers = drivers_df['years_of_driving_experience']
null_years = years_of_experience_for_drivers.isnull()
count_of_null_years = null_years.sum()
print(f"Количество пустых строк в 'years_of_driving_experience' для водителей: {count_of_null_years}")

Количество пустых строк в 'years_of_driving_experience' для водителей: 201464


In [ ]:
num_drivers = (df1['role'] == 'Водитель').sum()
print(f"Количество водителей в датасете: {num_drivers}")

Количество водителей в датасете: 2137177


In [ ]:
# не указан стаж вождения у 201 464 водителя или у 9,4 % водителей, эти данные удалять не будем, а заполним средним значением по полу.

In [ ]:
drivers_df = df1[df1['role'] == 'Водитель'].copy()

drivers_df['years_of_driving_experience'] = drivers_df['years_of_driving_experience'].replace('', np.nan)
average_experience_by_gender = drivers_df.groupby('gender')['years_of_driving_experience'].mean()
print("Средний стаж вождения по полу (только для водителей):")
print(average_experience_by_gender)

Средний стаж вождения по полу (только для водителей):
gender
Женский     9.597544
Мужской    15.764659
Name: years_of_driving_experience, dtype: float64


In [ ]:
df1['years_of_driving_experience'] = df1['years_of_driving_experience'].replace('', np.nan)
# Заполняем пустые значения для женщин-водителей средним значением
condition_female_driver_nan = (df1['role'] == 'Водитель') & \
                              (df1['gender'] == 'Женский') & \
                              (df1['years_of_driving_experience'].isna())

df1.loc[condition_female_driver_nan, 'years_of_driving_experience'] = 9

# Заполняем пустые значения для мужчин-водителей средним значением
condition_male_driver_nan = (df1['role'] == 'Водитель') & \
                            (df1['gender'] == 'Мужской') & \
                            (df1['years_of_driving_experience'].isna())

df1.loc[condition_male_driver_nan, 'years_of_driving_experience'] = 15

In [ ]:
# проверка
drivers_df = df1[df1['role'] == 'Водитель']
years_of_experience_for_drivers = drivers_df['years_of_driving_experience']
null_years = years_of_experience_for_drivers.isnull()
count_of_null_years = null_years.sum()
print(f"Количество пустых строк в 'years_of_driving_experience' для водителей: {count_of_null_years}")

Количество пустых строк в 'years_of_driving_experience' для водителей: 0


In [ ]:
df1['violations'].unique()

array(['Управление мопедом лицом моложе 16 лет, Управление ТС лицом, не имеющим права на управление ТС, Оставление места ДТП, Невыполнение требований обеспечения безопасности при начале движения',
       nan,
       'Несоблюдение условий, разрешающих движение транспорта задним ходом',
       ...,
       'Управление ТС лицом, не имеющим права на управление ТС, Нарушение правил применения мотошлема, Отказ водителя от прохождения медицинского освидетельствования на состояние опьянения, Неповиновение или сопротивление находящемуся при исполнении служебных обязанностей сотруднику правоохранительных органов или военнослужащему при остановке или задержании ТС',
       'Неправильный выбор дистанции, Управление ТС лицом, не имеющим права на управление ТС, Нарушение правил применения мотошлема, Оставление места ДТП, Отказ водителя, не имеющего права управления ТС либо лишенного права управления ТС, от прохождения медицинского освидетельствования на состояние опьянения (не применяется с 12.2020)'

In [ ]:
unique_count = df1['violations'].nunique()
unique_count

27513

In [ ]:
df1['violations_all'] = df1['violations']

Выведем новую колонку с инфомацией о состоянии участника ДТП (алк., нарк. опьянение)

In [ ]:
def extract_alc_info(road_str):
    if not isinstance(road_str, str):
        return 0

    road_lower = road_str.lower()

    if 'алкогол' in road_lower or 'состояние опьянения' in road_lower:
        return 1    # 'Алкогольное опьянение'
    elif 'наркот' in road_lower:
        return 1    # 'Наркотическое опьянение'
    else:
        return 0

df1['intoxication'] = df1['violations'].apply(extract_alc_info)
# 1 = участник ДТП в состоянии алкогольного/наркотического опьянения
# 0 = участник ДТП трезв

In [ ]:
alc_record = df1[df1['intoxication'] == 1]

num_alc_record = alc_record.shape[0]

print(f"Количество ДТП, где водитель в состоянии алк/нарк. опьянения: {num_alc_record}")

Количество ДТП, где водитель в состоянии алк/нарк. опьянения: 191464


Выведем новую колонку с инфомацией о водителе ТС, лишенном прав/не имеющим прав.

In [ ]:
def extract_dl_info(road_str):
    if not isinstance(road_str, str):
        return 0

    road_lower = road_str.lower()

    if 'лишенным права управления' in road_lower or 'не имеющим права на управление' in road_lower or 'не имеющим соответствующей категории' in road_lower:
        return 1
    else:
        return 0

df1['dl_revocation'] = df1['violations'].apply(extract_dl_info)

# 1 = езда без прав / водитель лишен прав
# 0 = водитель ТС с правами

In [ ]:
dl_revocation_record = df1[df1['dl_revocation'] == 1]

num_dl_revocation_record = dl_revocation_record.shape[0]

print(f"Количество ДТП, где водитель без прав: {num_dl_revocation_record}")

Количество ДТП, где водитель без прав: 139311


Выведем новую колонку с инфомацией о превышении скорости движения.

In [ ]:
def extract_speed_info(road_str):
    if not isinstance(road_str, str):
        return 0

    road_lower = road_str.lower()

    if 'несоответствие скорости' in road_lower or 'превышение установленной скорости' in road_lower:
        return 1
    else:
        return 0

df1['speeding'] = df1['violations'].apply(extract_speed_info)

# 1 = превышение скорости
# 0 = нет превышения скорости

In [ ]:
speeding_records = df1[df1['speeding'] == 1]
num_speeding_records = speeding_records.shape[0]

print(f"Количество ДТП с превышением скорости: {num_speeding_records}")

Количество ДТП с превышением скорости: 189543


In [ ]:
df1[['violations', 'intoxication', 'dl_revocation', 'speeding']].head(375)

,violations,intoxication,dl_revocation,speeding
0,"Управление мопедом лицом моложе 16 лет, Управл...",0,1,0
1,NaN,0,0,0
2,"Несоблюдение условий, разрешающих движение тра...",0,0,0
3,NaN,0,0,0
4,Нарушение правил пользования общественным тран...,0,0,0
...,...,...,...,...
373,NaN,0,0,0
374,"Несоблюдение бокового интервала, Управление ТС...",1,1,0
375,NaN,0,0,0
376,Нарушение требований сигналов светофора,0,0,0


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3418718 entries, 0 to 3547229
Data columns (total 12 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   accident_id                  int64  
 1   vehicle_id                   object 
 2   participant_id               object 
 3   role                         object 
 4   gender                       object 
 5   violations                   object 
 6   health_status                object 
 7   years_of_driving_experience  float64
 8   violations_all               object 
 9   intoxication                 int64  
 10  dl_revocation                int64  
 11  speeding                     int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 339.1+ MB


In [ ]:
df1['is_male'] = df1['gender'].apply(lambda x: 1 if x == 'Мужской' else 0)

In [ ]:
def auto_grouping_violations(value):
    if value is None or value != value:  # Проверка на NaN (float('nan') != float('nan'))
        return "Другие нарушения ПДД"
    if not isinstance(value, str):
        return value

    value_lower = value.lower()

    # Список условий в порядке приоритета
    conditions = [
        (["на полосу встречного движения", "во встречном направлении"], "Выезд на встречную полосу"),
        (["требований сигналов светофора", "сигналам регулирования"], "Нарушение требований сигналов светофора"),
        (["вне пешеходного перехода", "проезжей части в запрещённом месте", "через проезжую часть в неустановленном месте", "вдоль проезжей части", " ожидание маршрутного ТС на проезжей части", "игра на проезжей части", "на проезжей части без цели", "неожиданный выход из-за"], "Переход дороги вне пешеходного перехода"),
        (["преимущества в движении пешеходу", "правил проезда пешеходного"], "Нарушение правил проезда пешеходного перехода"),
        (["преимущества в движении ТС", "очередности проезда перекрестков"], "Несоблюдение очередности проезда"),
        (["разворот запрещён"], "Несоблюдение очередности проезда"),
        (["в возрасте до 7 лет без взрослого"], "Переход дороги без взрослого (до 7 лет)"),
        (["по проезжей части на скейтборде"], "Движение по проезжей части на скейте/самокате и т.д."),
        (["правил перестроения", "правил обгона"], "Нарушение правил обгона и перестроения"),
        (["стоянка на проезжей части", "правил остановки и стоянки"], "Нарушение правил остановки и стоянки"),
        (["при наличии неисправностей", "с техническими неисправност"], "Управление ТС с техническими неисправностями"),
        (["требований дорожных знаков", "требований линий разметки", "проезда остановок трамвая"], "Нарушение требований дорожных знаков и разметки"),
        (["выбор дистанции", "бокового интервала"], "Несоблюдение дистанции"),
        (["правил перевозки", "правил погрузки"], "Нарушение правил перевозки людей и грузов"),
        (["режима труда и отдыха", "сон за рулём", "состоянии переутомления", "нетрудоспособности"], "Управление ТС в состоянии переутомления"),
        #(["расположения ТС на проезжей"], "Нарушение правил расположения ТС на проезжей части"),
        (["ж/д", "на трамвайные пути"], "Неправильное пересечение ж/д переезда"),
        (["мобильной связью во время управления"], "Использование водителем мобильного телефона"),
        (["правил пользования внешними световыми"], "Езда без включенных фар"),
        (["мопедом лицом моложе 16", "велосипедом, гужевой"], "Управление мопедом до 16 лет"),
        (["другие нарушения"], "Другие нарушения ПДД"),
        #(["велосипед"], "Нарушение ПДД велосипедистом"),
        (["неправильная подача сигналов"], "Нарушение подачи сигналов"),
        #(["требований обеспечения безопасности при начале движения"], "Невыполнение требований обеспечения безопасности при начале движения"),
        (["опасное вождение"], "Опасное вождение"),
        (["ремней безопасности"], "Непристегнутый ремень безопасности"),
        (["применения мотошлема"], "Езда без мотошлема"),
        (["помех для водителя"], "Создание помех для водителя"),
        (["алкогол", "наркот", "состояние опьянения"], "Алкогольное/наркотическое опьянение"),
        (["лишенным права управления", "не имеющим права на управление", "не имеющим соответствующей категории"], "Езда без прав (в т.ч. лишенные прав)"),
        (["несоответствие скорости", "превышение установленной скорости"], "Превышение скорости")
    ]

    for keywords, category in conditions:
        # Проверяем, содержится ли хотя бы одно из ключевых слов в исходной строке
        if any(word in value_lower for word in keywords):
            return category

    # Если ни одно условие не найдено, возвращаем "Другие нарушения ПДД"
    return "Другие нарушения ПДД"

df1['violations'] = df1['violations_all'].apply(auto_grouping_violations)

In [ ]:
unique_count = df1['violations'].nunique()
unique_count

27

In [ ]:
# таким образом, сгруппировали нарушения ПДД - из 27 513 значений сделали 27 видов.

In [ ]:
df1[['violations', 'violations_all']].head(80)

,violations,violations_all
0,Управление мопедом до 16 лет,"Управление мопедом лицом моложе 16 лет, Управл..."
1,Другие нарушения ПДД,NaN
2,Другие нарушения ПДД,"Несоблюдение условий, разрешающих движение тра..."
3,Другие нарушения ПДД,NaN
4,Другие нарушения ПДД,Нарушение правил пользования общественным тран...
...,...,...
76,Превышение скорости,Несоответствие скорости конкретным условиям дв...
77,Другие нарушения ПДД,NaN
78,Нарушение правил проезда пешеходного перехода,"Нарушение правил проезда пешеходного перехода,..."
79,Другие нарушения ПДД,NaN


In [ ]:
df1['violations'].unique()

array(['Управление мопедом до 16 лет', 'Другие нарушения ПДД',
       'Выезд на встречную полосу', 'Несоблюдение дистанции',
       'Переход дороги вне пешеходного перехода',
       'Нарушение требований сигналов светофора',
       'Нарушение правил обгона и перестроения', 'Езда без мотошлема',
       'Несоблюдение очередности проезда',
       'Алкогольное/наркотическое опьянение',
       'Нарушение правил проезда пешеходного перехода',
       'Превышение скорости',
       'Управление ТС с техническими неисправностями',
       'Непристегнутый ремень безопасности',
       'Нарушение требований дорожных знаков и разметки',
       'Езда без прав (в т.ч. лишенные прав)',
       'Нарушение правил перевозки людей и грузов',
       'Движение по проезжей части на скейте/самокате и т.д.',
       'Неправильное пересечение ж/д переезда',
       'Управление ТС в состоянии переутомления',
       'Переход дороги без взрослого (до 7 лет)',
       'Создание помех для водителя',
       'Нарушение прави

In [ ]:
file_path = r'participants_all_regions_04-05.csv'
df1.to_csv(file_path, index=False)

Приступаем к обработке датасета с транспортными средствами

In [ ]:
df2 = pd.read_csv('vehicles_all_regions.csv', sep=';')

In [ ]:
accident_ids = df['id'].unique()
df2 = df2[df2['accident_id'].isin(accident_ids)].copy()

In [ ]:
df2.head()

,accident_id,vehicle_id,category,brand,model,color,year
0,2869171,2869171_1,Мопеды с двигателем внутреннего сгорания менее...,Прочие марки ТС,Прочие марки и модели ТС,Черный,2015.0
1,2320091,2320091_1,"С-класс (малый средний, компактный) до 4,3 м",TOYOTA,Corolla,Серый,2002.0
2,2321011,2321011_1,Одноэтажные длиной от 8 до 12 м,DAIMLER,Прочие модели DAIMLER,Многоцветный,1981.0
3,2576909,2576909_1,"В-класс (малый) до 3,9 м",HONDA,Torneo,Серый,2001.0
4,2576909,2576909_2,Мотоциклы,HONDA,Magna,Синий,1987.0


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2250426 entries, 0 to 2273036
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   accident_id  int64  
 1   vehicle_id   object 
 2   category     object 
 3   brand        object 
 4   model        object 
 5   color        object 
 6   year         float64
dtypes: float64(1), int64(1), object(5)
memory usage: 137.4+ MB


In [ ]:
df2.isnull().sum()

,0
accident_id,0
vehicle_id,0
category,0
brand,79223
model,79241
color,51084
year,89631


In [ ]:
df2['color'].unique()

array(['Черный', 'Серый', 'Многоцветный', 'Синий', 'Белый', 'Красный',
       'Иные цвета', 'Зеленый', 'Желтый', 'Фиолетовый', 'Оранжевый', nan,
       'Коричневый', 'Не заполнено'], dtype=object)

In [ ]:
color_counts = df2['color'].value_counts()
print("Количество по цветам:")
color_counts

Количество по цветам:


,count
color,
Белый,519015
Серый,382209
Иные цвета,343041
Черный,310147
Синий,206324
Красный,145869
Зеленый,107129
Желтый,50061
Коричневый,49096


In [ ]:
#заполним пропуски самыми популярными цветами ТС

In [ ]:
fill_colors = ['Серый', 'Белый', 'Черный']

indices_to_fill = df2[df2['color'].isna() | (df2['color'] == 'Не заполнено')].index

if not indices_to_fill.empty:
    random_colors = np.random.choice(fill_colors, size=len(indices_to_fill), replace=True)
    # Заполняем выбранные ячейки сгенерированными случайными цветами из самых популярных
    df2.loc[indices_to_fill, 'color'] = random_colors


In [ ]:
# Проверим распределение цветов после заполнения
color_counts = df2['color'].value_counts()
print("Количество по цветам:")
color_counts

Количество по цветам:


,count
color,
Белый,539009
Серый,402057
Иные цвета,343041
Черный,329731
Синий,206324
Красный,145869
Зеленый,107129
Желтый,50061
Коричневый,49096


In [ ]:
auto_counts = df2['brand'].value_counts()
print("Количество по маркам ТС:")
auto_counts.head(25)

Количество по маркам ТС:


,count
brand,
ВАЗ,478494
TOYOTA,191313
HYUNDAI,113024
KIA,92521
RENAULT,90094
NISSAN,86865
ГАЗ,77718
VOLKSWAGEN,74495
FORD,63610


In [ ]:
# заполним пропуски в колонке год ТС медианным значением года по таким же моделям ТС

In [ ]:
def fill_missing_years(df2):

    if not all(col in df2.columns for col in ['brand', 'model', 'year']):
        print("Ошибка: Отсутствуют необходимые столбцы 'brand', 'model' или 'year'.")
        return None

    if not pd.api.types.is_numeric_dtype(df2['year']):
        print("Ошибка: Столбец 'year' должен содержать числовые данные.")
        return None

    df2['year'] = pd.to_numeric(df2['year'], errors='coerce')
    nan_indices = df2[df2['year'].isnull()].index

    if nan_indices.empty:
        return df2
    medians = df2.groupby(['brand', 'model'])['year'].median()

    for (brand, model), year in medians.items():
        if pd.isna(year): continue

        brand_model_rows = df2[(df2['brand'] == brand) & (df2['model'] == model)]
        brand_model_rows_nan = brand_model_rows[brand_model_rows['year'].isnull()]
        if not brand_model_rows_nan.empty:
            df2.loc[brand_model_rows_nan.index, 'year'] = year

    return df2

df2 = fill_missing_years(df2)


In [ ]:
df2.isnull().sum()

,0
accident_id,0
vehicle_id,0
category,0
brand,79223
model,79241
color,0
year,71732


In [ ]:
# после обработки незаполненными остались 3,5% данных о ТС в части марки, модели и года выпуска. Так как размер пустыъ строк несущественный - удалим данные.

In [ ]:
df2 = df2.dropna(subset=['brand', 'model', 'year'])

In [ ]:
unique_count = df2['category'].nunique()
unique_count

98

In [ ]:
# укрупним категории ТС

In [ ]:
def extract_category(category_str):
    if not isinstance(category_str, str):
        return 'Иные ТС'

    category_lower = category_str.lower()

    if 'мопед' in category_lower or 'мотовело' in category_lower:
        return 'Мопеды'
    elif 'мото' in category_lower:
        return 'Мотоциклы'
    elif 'велосип' in category_lower:
        return 'Велосипеды'
    elif 'легков' in category_lower or 'класс' in category_lower:
        return 'Легковые авто'
    elif 'автобус' in category_lower or 'тролле' in category_lower or 'скорой' in category_lower  or 'грузов' in category_lower or 'минив' in category_lower:
        return 'Грузовые авто'
    elif 'одноэтаж' in category_lower or 'пожар' in category_lower or 'нефтепродук' in category_lower  or 'служебные' in category_lower or 'подъемник' in category_lower:
        return 'Грузовые авто'
    elif 'погрузчик' in category_lower or 'фургон' in category_lower or 'цистерн' in category_lower:
        return 'Грузовые авто'
    elif 'автобетон' in category_lower or 'автогрей' in category_lower or 'автокран' in category_lower or 'автоцемент' in category_lower or 'автоэвак' in category_lower:
        return 'Спецтехника'
    elif 'боев' in category_lower or 'бульд' in category_lower or 'строитель' in category_lower or 'специализ' in category_lower or 'спецтех' in category_lower:
        return 'Спецтехника'
    elif 'самосвал' in category_lower or 'самоходн' in category_lower or 'тягач' in category_lower or 'снегоубор' in category_lower or 'военн' in category_lower or 'эскават' in category_lower:
        return 'Спецтехника'
    elif 'ж/д' in category_lower or 'трамва' in category_lower:
        return 'Трамваи и ж/д составы'
    else:
        return 'Спецтехника'

df2['category'] = df2['category'].apply(extract_category)

In [ ]:
unique_count = df2['category'].nunique()
unique_count

7

In [ ]:
df2['category'].unique()

array(['Мопеды', 'Легковые авто', 'Грузовые авто', 'Мотоциклы',
       'Спецтехника', 'Велосипеды', 'Трамваи и ж/д составы'], dtype=object)

In [ ]:
# создадим столбец с возрастом ТС (дата ДТП - год выпуска ТС)

In [ ]:
# Объединение df и df2 по id

df2 = pd.merge(df2, df[['id', 'datetime']], left_on='accident_id', right_on='id', how='left')
# Удаляем столбец 'id' после объединения, так как он дублирует 'accident_id'
df2 = df2.drop('id', axis=1)

# Извлечение года из datetime
df2['year_of_accident'] = df2['datetime'].dt.year

# Расчет возраста ТС
df2['year2'] = df2['year_of_accident'] - df2['year']

# Обработка нулевого или отрицательного возраста
df2['year2'] = df2['year2'].apply(lambda x: 1 if x <= 0 else x)
df2['year2'] = df2['year2'].astype(int)

df2.head(20)

,accident_id,vehicle_id,category,brand,model,color,year,datetime,year_of_accident,year2
0,2869171,2869171_1,Мопеды,Прочие марки ТС,Прочие марки и модели ТС,Черный,2015.0,2023-05-24 19:30:00,2023,8
1,2320091,2320091_1,Легковые авто,TOYOTA,Corolla,Серый,2002.0,2015-09-16 13:00:00,2015,13
2,2321011,2321011_1,Грузовые авто,DAIMLER,Прочие модели DAIMLER,Многоцветный,1981.0,2017-08-25 15:30:00,2017,36
3,2576909,2576909_1,Легковые авто,HONDA,Torneo,Серый,2001.0,2021-05-29 19:40:00,2021,20
4,2576909,2576909_2,Мотоциклы,HONDA,Magna,Синий,1987.0,2021-05-29 19:40:00,2021,34
5,2547743,2547743_1,Легковые авто,HONDA,Civic,Серый,2003.0,2021-02-22 13:50:00,2021,18
6,2547743,2547743_2,Грузовые авто,ГАЗ,Прочие модели ГАЗ,Белый,2019.0,2021-02-22 13:50:00,2021,2
7,2758557,2758557_1,Легковые авто,TOYOTA,Kluger,Красный,2001.0,2022-09-17 10:30:00,2022,21
8,2758557,2758557_2,Легковые авто,MITSUBISHI,Colt,Серый,2003.0,2022-09-17 10:30:00,2022,19
9,2310675,2310675_1,Легковые авто,SKODA,Fabia,Белый,2012.0,2016-08-18 16:30:00,2016,4


In [ ]:
# убедившись, что возраст ТС посчитан корректно, удаляем лишние колонки
df2 = df2.drop(['year','datetime','year_of_accident'], axis=1)

In [ ]:
df2 = df2.rename(columns={'year2': 'year'})

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2171173 entries, 0 to 2171172
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   accident_id  int64 
 1   vehicle_id   object
 2   category     object
 3   brand        object
 4   model        object
 5   color        object
 6   year         int64 
dtypes: int64(2), object(5)
memory usage: 116.0+ MB


In [ ]:
df2.head(20)

,accident_id,vehicle_id,category,brand,model,color,year
0,2869171,2869171_1,Мопеды,Прочие марки ТС,Прочие марки и модели ТС,Черный,8
1,2320091,2320091_1,Легковые авто,TOYOTA,Corolla,Серый,13
2,2321011,2321011_1,Грузовые авто,DAIMLER,Прочие модели DAIMLER,Многоцветный,36
3,2576909,2576909_1,Легковые авто,HONDA,Torneo,Серый,20
4,2576909,2576909_2,Мотоциклы,HONDA,Magna,Синий,34
5,2547743,2547743_1,Легковые авто,HONDA,Civic,Серый,18
6,2547743,2547743_2,Грузовые авто,ГАЗ,Прочие модели ГАЗ,Белый,2
7,2758557,2758557_1,Легковые авто,TOYOTA,Kluger,Красный,21
8,2758557,2758557_2,Легковые авто,MITSUBISHI,Colt,Серый,19
9,2310675,2310675_1,Легковые авто,SKODA,Fabia,Белый,4


In [ ]:
df = df[df['id'] != 2947689]   #при открытии файла выяснилось, что строка с данным индектом пустая - удалим ее

In [ ]:
# удалим в df запсии о ДТП, удаленные при обработке df1
accident_ids = df1['accident_id'].unique()
df = df[df['id'].isin(accident_ids)].copy()

In [ ]:
# удалим в df запсии о ДТП, удаленные при обработке df2
accident_ids = df2['accident_id'].unique()
df = df[df['id'].isin(accident_ids)].copy()

In [ ]:
# теперь удалим удаленные в df строки в df1, df2

In [ ]:
accident_ids = df['id'].unique()
df1 = df1[df1['accident_id'].isin(accident_ids)].copy()
df2 = df2[df2['accident_id'].isin(accident_ids)].copy()

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2165569 entries, 0 to 2171172
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   accident_id  int64 
 1   vehicle_id   object
 2   category     object
 3   brand        object
 4   model        object
 5   color        object
 6   year         int64 
dtypes: int64(2), object(5)
memory usage: 132.2+ MB


In [ ]:
file_path = r'accidents_all_regions_07-05.csv'
df.to_csv(file_path, index=False, encoding='utf-8')

In [ ]:
file_path = r'participants_all_regions_06-05.csv'
df1.to_csv(file_path, index=False, encoding='utf-8')

In [ ]:
file_path = r'vehicles_all_regions_06-05.csv'
df2.to_csv(file_path, index=False, encoding='utf-8')

In [ ]:
# Обработка датасета с количеством населения РФ.

In [ ]:
df3 = pd.read_csv('data.csv')

In [ ]:
df3['population_region'] = df3.groupby('region')['population'].transform('sum')

In [ ]:
df3 = df3.drop(columns=['id', 'municipality', 'type', 'settlement', 'children', 'population', 'oktmo', 'dadata', 'rosstat'])

In [ ]:
df3= df3.drop_duplicates(subset=['region'])

In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, 0 to 40493
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   region             86 non-null     object 
 1   latitude_dms       86 non-null     object 
 2   longitude_dms      86 non-null     object 
 3   latitude_dd        86 non-null     float64
 4   longitude_dd       86 non-null     float64
 5   population_region  86 non-null     int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 4.7+ KB


In [ ]:
df3.head()

,region,latitude_dms,longitude_dms,latitude_dd,longitude_dd,population_region
0,Орловская область,53.22.07,035.54.36,53.368611,35.910000,752315
1,Республика Крым,44.35.45,034.20.27,44.595833,34.340833,2496928
2,Липецкая область,53.15.46,039.30.14,53.262778,39.503889,1143940
3,Тверская область,56.54.20,033.25.09,56.905556,33.419167,1303663
4,Томская область,58.38.12,082.02.40,58.636667,82.044444,1077442


In [ ]:
file_path = r'regions.csv'
df3.to_csv(file_path, index=False, encoding='utf-8')